### Load Dataset Files

In [0]:
from google.colab import drive

In [10]:
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
project_path = "/content/drive/My Drive/Dog/data/"

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

In [0]:
import pandas as pd

In [20]:
labels = pd.read_csv('./labels.csv')
labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [29]:
labels.columns

Index([u'id', u'breed'], dtype='object')

In [31]:
breed_count = labels['breed'].value_counts()

breed_count.head()

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

In [32]:
breed_count.shape

(120,)

### One-hot encodings of labels

In [0]:
import numpy as np
targets = pd.Series(labels['breed'])
one_hot = pd.get_dummies(targets, sparse = True)
one_hot_labels = np.asarray(one_hot)

In [36]:
one_hot_labels[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)

Or you can use below code also for creating one hot encodings

In [37]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(labels['breed'])
print(integer_encoded)

[19 37 85 ...  3 75 28]


In [38]:
import keras
y = keras.utils.to_categorical(integer_encoded, num_classes=None, dtype='int')

Using TensorFlow backend.


In [39]:
y[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [0]:
img_rows=128
img_cols=128
num_channel=1

In [43]:
from tqdm import tqdm
import cv2
x_feature = []
y_feature = []

i = 0 # initialisation
for f, img in tqdm(labels.values): # f for format ,jpg
    train_img = cv2.imread('./train/{}.jpg'.format(f),0)
    label = one_hot_labels[i]
    train_img_resize = cv2.resize(train_img, (img_rows, img_cols)) 
    x_feature.append(train_img_resize)
    y_feature.append(label)
    i += 1

100%|██████████| 10222/10222 [00:13<00:00, 745.40it/s]


In [44]:
x_train_data = np.array(x_feature, np.float32) / 255.   # /= 255 for normolisation
print (x_train_data.shape)
x_train_data = np.expand_dims(x_train_data, axis = 3) # for keras to given input to Conv2D layer
print (x_train_data.shape)


(10222, 128, 128)
(10222, 128, 128, 1)


In [45]:
y_train_data = np.array(y_feature)

y_train_data.shape

(10222, 120)

### Split the training and validation data here

In [46]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train_data, y_train_data, test_size=0.2, random_state=2)
print (x_train.shape)
print (x_val.shape)

(8177, 128, 128, 1)
(2045, 128, 128, 1)


### Test features

In [47]:
submission = pd.read_csv('sample_submission.csv')
test_img = submission['id']
submission.head()

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
1,00102ee9d8eb90812350685311fe5890,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
2,0012a730dfa437f5f3613fb75efcd4ce,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333
3,001510bc8570bbeee98c8d80c8a95ec1,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333

In [48]:
x_test_feature = []

i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 0)
    img_resize = cv2.resize(img, (img_rows, img_cols)) 
    x_test_feature.append(img_resize)

100%|██████████| 10357/10357 [00:12<00:00, 824.41it/s]


In [49]:
x_test_data = np.array(x_test_feature, np.float32) / 255. 
print (x_test_data.shape)
x_test_data = np.expand_dims(x_test_data, axis = 3)
print (x_test_data.shape)

(10357, 128, 128)
(10357, 128, 128, 1)


In [0]:
from keras.models import Sequential  # initial NN
from keras.layers import Dense, Dropout # construct each layer
from keras.layers import Conv2D # swipe across the image by 1
from keras.layers import MaxPooling2D # swipe across by pool size
from keras.layers import Flatten, GlobalAveragePooling2D

In [51]:
model = Sequential()

model.add(Conv2D(32, (5,5), activation ='relu', input_shape = (img_rows, img_cols, num_channel))) 
model.add(MaxPooling2D(pool_size=3))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=64, kernel_size=4, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
#model.add(Dropout(0.2))

model.add(Conv2D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.8))

model.add(Conv2D(filters=128, kernel_size=2, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=2))
model.add(Dropout(0.8))

model.add(Flatten()) 
# fully connected layer 
model.add(Dense(units = 500, activation = 'relu')) 
model.add(Dropout(0.8))
# output layer
model.add(Dense(units = 120, activation = 'softmax')) 

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [52]:
from keras.optimizers import Adam

optimizer = Adam(lr=0.01)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"]) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 124, 124, 32)      832       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 41, 41, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 64)        32832     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 20, 20, 128)       73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 10, 128)       0         
__________

In [53]:
batch_size = 128
nb_epochs = 20
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=nb_epochs, 
                    validation_data=(x_val, y_val),
                    initial_epoch=0)

Instructions for updating:
Use tf.cast instead.
Train on 8177 samples, validate on 2045 samples
Epoch 1/20
8177/8177 [==============================] - 9s 1ms/step - loss: 4.8810 - acc: 0.0092 - val_loss: 4.7889 - val_acc: 0.0088
Epoch 2/20
8177/8177 [==============================] - 4s 453us/step - loss: 4.7820 - acc: 0.0113 - val_loss: 4.7889 - val_acc: 0.0117
Epoch 3/20
8177/8177 [==============================] - 4s 450us/step - loss: 4.7788 - acc: 0.0108 - val_loss: 4.7900 - val_acc: 0.0088
Epoch 4/20
8177/8177 [==============================] - 4s 445us/step - loss: 4.7774 - acc: 0.0138 - val_loss: 4.7911 - val_acc: 0.0117
Epoch 5/20
8177/8177 [==============================] - 4s 441us/step - loss: 4.7774 - acc: 0.0117 - val_loss: 4.7921 - val_acc: 0.0117
Epoch 6/20
8177/8177 [==============================] - 4s 441us/step - loss: 4.7769 - acc: 0.0117 - val_loss: 4.7929 - val_acc: 0.0117
Epoch 7/20
8177/8177 [==============================] - 4s 444us/step - loss: 4.7771 - acc

### Using Data Augmentation

In [0]:
from keras.preprocessing.image import img_to_array,load_img,ImageDataGenerator
from keras.layers import ZeroPadding2D

train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen=ImageDataGenerator(rescale=1./255)

In [0]:
training_set=train_datagen.flow(x_train,y=y_train,batch_size=32)
testing_set=test_datagen.flow(x_val,y=y_val,batch_size=32)

In [56]:
model=Sequential()

model.add(ZeroPadding2D((1,1),input_shape=(128,128,1)))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(ZeroPadding2D(padding=(1,1)))

model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(32,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(500,activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(120,activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d_1 (ZeroPaddin (None, 130, 130, 1)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 128, 128, 32)      320       
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 130, 130, 32)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 128, 128, 32)      9248      
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 126, 126, 32)      9248      
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 63, 63, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 61, 61, 64)        18496     
__________

In [57]:
history=model.fit_generator(training_set,
                      steps_per_epoch = 500,
                      validation_data = testing_set,
                      validation_steps = 4,
                      epochs = 20,
                      verbose = 1)

Epoch 1/20
500/500 [==============================] - 33s 66ms/step - loss: 4.7831 - acc: 0.0103 - val_loss: 4.8086 - val_acc: 0.0078
Epoch 2/20
500/500 [==============================] - 30s 60ms/step - loss: 4.7777 - acc: 0.0107 - val_loss: 4.7768 - val_acc: 0.0234
Epoch 3/20
500/500 [==============================] - 31s 61ms/step - loss: 4.7761 - acc: 0.0111 - val_loss: 4.8241 - val_acc: 0.0078
Epoch 4/20
500/500 [==============================] - 30s 61ms/step - loss: 4.7767 - acc: 0.0122 - val_loss: 4.7970 - val_acc: 0.0000e+00
Epoch 5/20
500/500 [==============================] - 30s 61ms/step - loss: 4.7764 - acc: 0.0108 - val_loss: 4.7867 - val_acc: 0.0156
Epoch 6/20
500/500 [==============================] - 31s 61ms/step - loss: 4.7756 - acc: 0.0113 - val_loss: 4.7845 - val_acc: 0.0234
Epoch 7/20
500/500 [==============================] - 31s 61ms/step - loss: 4.7765 - acc: 0.0119 - val_loss: 4.7853 - val_acc: 0.0156
Epoch 8/20
500/500 [==============================] - 31s 

### Using Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input


# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

In [61]:
from tqdm import tqdm
import cv2
x_feature = []
y_feature = []

i = 0 # initialisation
for f, img in tqdm(labels.values): # f for format ,jpg
    train_img = cv2.imread('./train/{}.jpg'.format(f),1)
    label = one_hot_labels[i]
    train_img_resize = cv2.resize(train_img, (img_rows, img_cols)) 
    x_feature.append(train_img_resize)
    y_feature.append(label)
    i += 1

100%|██████████| 10222/10222 [00:24<00:00, 424.55it/s]


### Get Image embedding from the pre-trained model and use that as input to present model

In [62]:
from keras.preprocessing import image

X_arr = []
# X_arr = base_model.predict(preprocess_input(img_to_array(image.load_img('./train/'+labels['id'][0]+'.jpg', target_size=[128,128]))))

for id, label in tqdm(labels.values):
    X_arr.append(base_model.predict(preprocess_input(np.expand_dims(img_to_array(image.load_img('./train/'+id+'.jpg', target_size=[128,128])), axis=0)))[0])
X = pd.DataFrame(X_arr)

100%|██████████| 10222/10222 [01:18<00:00, 130.28it/s]


In [0]:
from keras.layers import Activation
model = Sequential([
    Dense(1024, input_shape=(512,)),
    Activation('relu'),
    Dense(256, input_shape=(512,)),
    Activation('relu'),
    Dense(120),
    Activation('softmax'),
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
X_train = X
y_train = y_feature

In [65]:
model.fit(X_train, np.asarray(y_train), epochs=100, batch_size=100, verbose=True)

Epoch 1/100
10222/10222 [==============================] - 1s 95us/step - loss: 4.7309 - acc: 0.1597
Epoch 2/100
10222/10222 [==============================] - 1s 55us/step - loss: 2.1690 - acc: 0.4324
Epoch 3/100
10222/10222 [==============================] - 1s 54us/step - loss: 1.5251 - acc: 0.5729
Epoch 4/100
10222/10222 [==============================] - 1s 59us/step - loss: 1.0706 - acc: 0.6912
Epoch 5/100
10222/10222 [==============================] - 1s 60us/step - loss: 0.7384 - acc: 0.7871
Epoch 6/100
10222/10222 [==============================] - 1s 61us/step - loss: 0.4743 - acc: 0.8593
Epoch 7/100
10222/10222 [==============================] - 1s 57us/step - loss: 0.2928 - acc: 0.9183
Epoch 8/100
10222/10222 [==============================] - 1s 55us/step - loss: 0.1432 - acc: 0.9662
Epoch 9/100
10222/10222 [==============================] - 1s 55us/step - loss: 0.0707 - acc: 0.9877
Epoch 10/100
10222/10222 [==============================] - 1s 54us/step - loss: 0.0384 - a